In [91]:
import sys
#sys.path.insert(0, "../..")

from typing import Callable, Protocol, Dict, Optional, Iterator, List, Tuple

import gin
from pathlib import Path
import numpy as np
import math

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Optimizer
from torchinfo import summary
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#from src.data import make_dataset
#from src.models import metrics
#from src.models import train_model # om de functie train_loop binnen te halen

# ------------------------------------------------------------------------------- #
## Alle imports benodigd voor de functie train_loop uit train_model.py 

import tensorflow as tf  # noqa: F401

# needed to make summarywriter load without error
from loguru import logger
from numpy import Inf
from ray import tune


from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

#from src.models.metrics import Metric
#from src.typehinting import GenericModel
#from src.data import data_tools

In [92]:
## benodigde functie uitmake_dataset.py

#@gin.configurable
def get_MNIST(  # noqa: N802
    data_dir: Path, batch_size: int
) -> Tuple[DataLoader, DataLoader]:

    training_data = datasets.FashionMNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

In [56]:
## Benodigde funties uit metrics.py

Tensor = torch.Tensor

class Metric:
    def __repr__(self) -> str:
        raise NotImplementedError

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        raise NotImplementedError

class Accuracy(Metric):
    def __repr__(self) -> str:
        return "Accuracy"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        return (yhat.argmax(dim=1) == y).sum() / len(yhat)


In [57]:
class GenericModel(Protocol):
    train: Callable
    eval: Callable
    parameters: Callable

    def __call__(self, *args, **kwargs) -> torch.Tensor:
        pass


In [58]:
def dir_add_timestamp(log_dir: Optional[Path] = None) -> Path:
    if log_dir is None:
        log_dir = Path(".")
    log_dir = Path(log_dir)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    log_dir = log_dir / timestamp
    logger.info(f"Logging to {log_dir}")
    if not log_dir.exists():
        log_dir.mkdir(parents=True)
    return log_dir

In [59]:
## Definieren functie train_loop

#@gin.configurable
def trainloop(
    epochs: int,
    model: GenericModel,
    optimizer: torch.optim.Optimizer,
    learning_rate: float,
    loss_fn: Callable,
    metrics: List[Metric],
    train_dataloader: Iterator,
    test_dataloader: Iterator,
    log_dir: Path,
    train_steps: int,
    eval_steps: int,
    patience: int = 10,
    factor: float = 0.9,
    tunewriter: bool = False,
) -> GenericModel:
    
    optimizer_: torch.optim.Optimizer = optimizer(
        model.parameters(), lr=learning_rate
    )  # type: ignore

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_,
        factor=factor,
        patience=patience,
    )

    if not tunewriter:
        log_dir = data_tools.dir_add_timestamp(log_dir)
        writer = SummaryWriter(log_dir=log_dir)
        write_gin(log_dir, gin.config_str())

        images, _ = next(iter(train_dataloader))
        if len(images.shape) == 4:
            grid = make_grid(images)
            writer.add_image("images", grid)
        writer.add_graph(model, images)

    for epoch in tqdm(range(epochs)):
        train_loss = trainbatches(
            model, train_dataloader, loss_fn, optimizer_, train_steps
        )

        metric_dict, test_loss = evalbatches(
            model, test_dataloader, loss_fn, metrics, eval_steps
        )

        scheduler.step(test_loss)

        if tunewriter:
            tune.report(
                iterations=epoch,
                train_loss=train_loss,
                test_loss=test_loss,
                **metric_dict,
            )
        else:
            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            for m in metric_dict:
                writer.add_scalar(f"metric/{m}", metric_dict[m], epoch)
            lr = [group["lr"] for group in optimizer_.param_groups][0]
            writer.add_scalar("learning_rate", lr, epoch)
            metric_scores = [f"{v:.4f}" for v in metric_dict.values()]
            logger.info(
                f"Epoch {epoch} train {train_loss:.4f} test {test_loss:.4f} metric {metric_scores}"  # noqa E501
            )

    return model

In [60]:
def count_parameters(model: GenericModel) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
sys.path

In [74]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cpu device


In [75]:
## Opbouw van het model

#@gin.configurable
class CNN(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            #nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2),
            
            #nn.Conv2d(filter2, 32, kernel_size=kernel_size, stride=1, padding=0),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(3136, 1568),
            nn.ReLU(),

            nn.Linear(1568, 784),
            nn.ReLU(),
            
            nn.Linear(784, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [ ]:
gin.parse_config_file("model.gin")

In [101]:
# 10 = aantal classes, 3 = kernel_size, 32 = grootte filter 1, 32 = grootte  filter2

model = CNN(10, 3, 16,16).to(device)
print(model)
summary(model, input_size=(64, 1, 28, 28))

CNN(
  (convolutions): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=1568, bias=True)
    (2): ReLU()
    (3): Linear(in_features=1568, out_features=784, bias=True)
    (4): ReLU()
    (5): Linear(in_features=784, out_features=10, bias=True)
  )
)


Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [64, 10]                  --
├─Sequential: 1-1                        [64, 16, 14, 14]          --
│    └─Conv2d: 2-1                       [64, 16, 28, 28]          160
│    └─ReLU: 2-2                         [64, 16, 28, 28]          --
│    └─MaxPool2d: 2-3                    [64, 16, 14, 14]          --
├─Sequential: 1-2                        [64, 10]                  --
│    └─Flatten: 2-4                      [64, 3136]                --
│    └─Linear: 2-5                       [64, 1568]                4,918,816
│    └─ReLU: 2-6                         [64, 1568]                --
│    └─Linear: 2-7                       [64, 784]                 1,230,096
│    └─ReLU: 2-8                         [64, 784]                 --
│    └─Linear: 2-9                       [64, 10]                  7,850
Total params: 6,156,922
Trainable params: 6,156,922
Non-trainable p

In [77]:
count_parameters(model)

6156922

In [102]:
datadir =  Path("/home/admindme/code/ML22_opdracht1/data/raw/FashionMNIST")
train_dataloader, test_dataloader = get_MNIST(datadir, 64)
len(train_dataloader), len(test_dataloader)

In [108]:
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = Accuracy()

In [109]:
filters = [8]
kernels = [3]
lr = 1e-3

# 4 required positional arguments: 'optimizer', 'learning_rate', 'loss_fn', and 'log_dir'
optimizer = Adam
learning_rate = 0.001
loss_fn = 
log_dir = 

for filter in filters:
    for kernel in kernels:
        #gin.bind_parameter("trainloop.learning_rate", lr)
        #gin.bind_parameter("CNN.kernel_size", kernel)
        
        model = CNN()
        #model = CNN(
        #    num_classes=10,
        #    kernel_size=kernel,
        #    filter1=filter,
        #    filter2=filter*2
        #)
            
        model = trainloop(
            epochs=3,
            model=model,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            train_steps=len(train_dataloader),
            eval_steps=150,
        )

TypeError: __init__() missing 4 required positional arguments: 'num_classes', 'kernel_size', 'filter1', and 'filter2'

In [61]:
sys.path.remove('../..')
sys.path.remove('/home/admindme/code/ML22_opdracht1/notebooks')
sys.path.insert(0, '/home/admindme/code/ML22_opdracht1/src')
sys.path.insert(0, '../src')

# Changing the CWD
os.chdir('../src')
#os.chdir('../notebooks')